## Importing Libraries

In [1]:
import spacy 
import textacy
nlp = spacy.load('en_core_web_sm')

In [2]:
from string import punctuation
from collections import defaultdict
from nltk.stem.snowball import SnowballStemmer as Stemmer

In [3]:
from spacy.lang.en.stop_words import STOP_WORDS

In [4]:
from scipy.spatial.distance import pdist
import numpy as np

In [57]:
from scipy.cluster.hierarchy import linkage, fcluster
import networkx as nx
from itertools import combinations

## Text Used
A blockchain,originally block chain, is a growing list of records, called blocks, which are linked using cryptography.Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data (generally represented as a merkle tree root hash). By design, a blockchain is resistant to modification of the data. It is "an open, distributed ledger that can record transactions between two parties efficiently and in a verifiable and permanent way". For use as a distributed ledger, a blockchain is typically managed by a peer-to-peer network collectively adhering to a protocol for inter-node communication and validating new blocks. Once recorded, the data in any given block cannot be altered retroactively without alteration of all subsequent blocks, which requires consensus of the network majority. Although blockchain records are not unalterable, blockchains may be considered secure by design and exemplify a distributed computing system with high Byzantine fault tolerance. Decentralized consensus has therefore been claimed with a blockchain. Blockchain was invented by Satoshi Nakamoto in 2008 to serve as the public transaction ledger of the cryptocurrency bitcoin. 
The invention of the blockchain for bitcoin made it the first digital currency to solve the double-spending problem without the need of a trusted authority or central server. The bitcoin design has inspired other applications, and blockchains which are readable by the public are widely used by cryptocurrencies. Private blockchains have been proposed for business use. Some marketing of blockchains has been called "snake oil".

In [6]:
text = "A blockchain,originally block chain, is a growing list of records, called blocks, which are linked using cryptography. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data (generally represented as a merkle tree root hash). By design, a blockchain is resistant to modification of the data. It is an open, distributed ledger that can record transactions between two parties efficiently and in a verifiable and permanent way. For use as a distributed ledger, a blockchain is typically managed by a peer-to-peer network collectively adhering to a protocol for inter-node communication and validating new blocks. Once recorded, the data in any given block cannot be altered retroactively without alteration of all subsequent blocks, which requires consensus of the network majority. Although blockchain records are not unalterable, blockchains may be considered secure by design and exemplify a distributed computing system with high Byzantine fault tolerance. Decentralized consensus has therefore been claimed with a blockchain. Blockchain was invented by Satoshi Nakamoto in 2008 to serve as the public transaction ledger of the cryptocurrency bitcoin. The invention of the blockchain for bitcoin made it the first digital currency to solve the double-spending problem without the need of a trusted authority or central server. The bitcoin design has inspired other applications, and blockchains which are readable by the public are widely used by cryptocurrencies. Private blockchains have been proposed for business use. Some marketing of blockchains has been called snake oil."              

In [7]:
doc = nlp(u"A blockchain,originally block chain, is a growing list of records, called blocks, which are linked using cryptography. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data (generally represented as a merkle tree root hash). By design, a blockchain is resistant to modification of the data. It is an open, distributed ledger that can record transactions between two parties efficiently and in a verifiable and permanent way. For use as a distributed ledger, a blockchain is typically managed by a peer-to-peer network collectively adhering to a protocol for inter-node communication and validating new blocks. Once recorded, the data in any given block cannot be altered retroactively without alteration of all subsequent blocks, which requires consensus of the network majority. Although blockchain records are not unalterable, blockchains may be considered secure by design and exemplify a distributed computing system with high Byzantine fault tolerance. Decentralized consensus has therefore been claimed with a blockchain. Blockchain was invented by Satoshi Nakamoto in 2008 to serve as the public transaction ledger of the cryptocurrency bitcoin. The invention of the blockchain for bitcoin made it the first digital currency to solve the double-spending problem without the need of a trusted authority or central server. The bitcoin design has inspired other applications, and blockchains which are readable by the public are widely used by cryptocurrencies. Private blockchains have been proposed for business use. Some marketing of blockchains has been called snake oil.")       

In [69]:

class Candidate(object):
    """ The keyphrase candidate data structure. """

    def __init__(self):

        self.surface_forms = []
        """ the surface forms of the candidate. """

        self.offsets = []
        """ the offsets of the surface forms. """

        self.sentence_ids = []
        """ the sentence id of each surface form. """

        self.pos_patterns = []
        """ the Part-Of-Speech patterns of the candidate. """

        self.lexical_form = []
        """ the lexical form of the candidate. """



## Helper Functions

In [70]:
def vectorize_candidates(candidates , topics):
    """Vectorize the keyphrase candidates.
    Returns:
    C (list): the list of candidates.
    X (matrix): vectorized representation of the candidates.
    """
    # build the vocabulary, i.e. setting the vector dimensions
    dim = set([])
    # for k, v in self.candidates.iteritems():
    # iterate Python 2/3 compatible
     
    for (k, v) in candidates.items():
        for w in v.lexical_form:
            dim.add(w)
    dim = list(dim)
    # vectorize the candidates Python 2/3 + sort for random issues
    C = list(candidates) #.keys()
    C.sort()

    X = np.zeros((len(C), len(dim)))
    for i, k in enumerate(C):
        for w in candidates[k].lexical_form:
            X[i, dim.index(w)] += 1

    return C , X


In [71]:
def topic_clustering(candidates , threshold=0.74, method='average'):
    """Clustering candidates into topics.
        Args:
        threshold (float): the minimum similarity for clustering, defaults
        to 0.74, i.e. more than 1/4 of stem overlap similarity.
        method (str): the linkage method, defaults to average.
    """
    topic = []
    # handle document with only one candidate
    if len(candidates) == 1:
        topics.append([list(candidates)[0]])
        return topic


    # vectorize the candidates
    candidates, X = vectorize_candidates(candidates , topic )

    # compute the distance matrix
    Y = pdist(X, 'jaccard')
    
    # compute the clusters
    Z = linkage(Y, method=method)
    
    # form flat clusters
    clusters = fcluster(Z, t=threshold, criterion='distance')

    # for each topic identifier
    for cluster_id in range(1, max(clusters)+1):
        topic.append([candidates[j] for j in range(len(clusters))
                    if clusters[j] == cluster_id])
    
    return topic


In [72]:
def build_topic_graph(graph  , topics , candidates ):
    """Build topic graph."""
   
    # adding the nodes to the graph
    graph.add_nodes_from(range(len(topics)))

    # loop through the topics to connect the nodes
    for i, j in combinations(range(len(topics)), 2):
        graph.add_edge(i, j, weight=0.0)
        for c_i in topics[i]:
            for c_j in topics[j]:
                for p_i in candidates[c_i].offsets:
                    for p_j in candidates[c_j].offsets:
                        gap = abs(p_i - p_j)
                        if p_i < p_j:
                            gap -= len(candidates[c_i].lexical_form)-1
                        if p_j < p_i:
                            gap -= len(candidates[c_j].lexical_form)-1
                        graph[i][j]['weight'] += 1.0 / gap

                                
                    
    return graph

In [73]:
def is_redundant(candidates, candidate, prev, mininum_length=1):
    """ Test if one candidate is redundant with respect to a list of already
    selected candidates. A candidate is considered redundant if it is
    included in another candidate that is ranked higher in the list.
    Args:
    candidate (str): the lexical form of the candidate.
    prev (list): the list of already selected candidates (lexicalforms).
    mininum_length (int): minimum length (in words) of the candidate
    to be considered, defaults to 1.
    """

    # get the tokenized lexical form from the candidate
    candidate = candidates[candidate].lexical_form

    # only consider candidate greater than one word
    if len(candidate) < mininum_length:
        return False

    # get the tokenized lexical forms from the selected candidates
    prev = [candidates[u].lexical_form for u in prev]

    # loop through the already selected candidates
    for prev_candidate in  prev:
        for i in range(len(prev_candidate)-len(candidate)+1):
            if candidate == prev_candidate[i:i+len(candidate)]:
                return True
    return False


In [74]:
def _is_alphanum(word, valid_punctuation_marks = '-'):
    """Check if a word is valid, i.e. it contains only alpha-numeric
    characters and valid punctuation marks.
    Args:
        word (string): a word.
        valid_punctuation_marks (str): punctuation marks that are valid
        for a candidate, defaults to '-'.
    """
    for punct in valid_punctuation_marks.split():
        word = word.replace(punct, '')
    return word.isalnum()

In [75]:
def add_candidate( candidates ,  words, offset, sentence_id):
        """ Add a keyphrase candidate to the candidates container.
            Args:
                words (list): the words (surface form) of the candidate.
                stems (list): the stemmed words of the candidate.
                pos (list): the Part-Of-Speeches of the words in the candidate.
                offset (int): the offset of the first word of the candidate.
                sentence_id (int): the sentence id of the candidate.
        """
        stemmer='porter'
        stems = []
        for j, word in enumerate(words):
            stems.append(Stemmer(stemmer).stem(str(word)))
    
        
        # build the lexical (canonical) form of the candidate using stems
        lexical_form = ' '.join(stems)

        # add/update the surface forms
        a = []
        for j , word in enumerate(words):
            a.append(str(word))
        candidates[lexical_form].surface_forms.append(a)
        # add/update the lexical_form
        candidates[lexical_form].lexical_form = stems
      
        # add/update the POS patterns
        pos = []
        for j , word in enumerate(words):
            pos.append(word.pos_)
            
        candidates[lexical_form].pos_patterns.append(pos)

        # add/update the offsets
        candidates[lexical_form].offsets.append(offset)
        
        # add/update the sentence ids
        candidates[lexical_form].sentence_ids.append(sentence_id)


## TopicRank 

In [76]:
def TopicRank(doc, nkeyterms = 10 , pos = None):
    
    ##STEP  1 : candidate_selection
    if pos is None:
        pos_ = set(['NOUN', 'PROPN', 'ADJ'])  
     
        #### select sequence of adjectives and nouns
    
    sentences = []  
    t = []
    k = 0
    candidates = defaultdict(Candidate)
    # loop through the sentences
    for i in doc.sents:
        l = 0 
        for _ in i :
            l = l+1
        for j in i : 
            t.append(str(j))
        sentences.append(list(t))
        t = []
        a = sentences
        
        # compute the offset shift for the sentence
        shift = sum([ len(s) for s in a[0:k]])
        k = k + 1
        
        # container for the sequence (defined as list of offsets)
        seq  = [ ]
        # loop through the tokens
        for j in enumerate(i):
        
            # add candidate offset in sequence and continue if not last word
            
            if j[1].pos_ in pos_:
                seq.append(j[0])
                if j[0] < l:
                    continue
        
            # container for the sequence (defined as list of offsets)
            if seq :
                        # bias for candidate in last position within sentence
                bias = 0                
                if j[0] == l :
                    bias = 1
                words = i[seq[0]:seq[-1]+1]
                
                
    
                # add the ngram to the candidate container
                add_candidate(candidates , words = i[seq[0]:seq[-1]+1],
                                offset= shift + j[0] - len(seq) + bias,
                                sentence_id=k)
            
            # flush sequence container
            seq = []      
         

    
    ## STEP 2 : CANDIDATE FILTERING : ( # filter candidates containing stopwords or punctuation marks)
    
    # initialize stoplist list if not provided
    stoplist = list(STOP_WORDS)
    
    stoplist_ = list(punctuation) + ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-','-rsb-'] + stoplist
    mininum_length = 3 
    mininum_word_size = 2 
    valid_punctuation_marks = '-' 
    maximum_word_number = 5 
    only_alphanum = True 
    pos_blacklist = []
        
    # loop throught the candidates    
    y = 1
    for k in list(candidates):
        y = y + 1
        # get the candidate
        v = candidates[k]
        
        # get the words from the first occurring surface form
        words_lower = [u.lower() for u in v.surface_forms[0]]

        
        # discard if words are in the stoplist
        if set(words).intersection(stoplist_):
            del candidates[k]

        # discard if tags are in the pos_blacklist
        elif set(v.pos_patterns[0]).intersection(pos_blacklist):
            del candidates[k]

        # discard if containing tokens composed of only punctuation

        elif any([set(u).issubset(set(punctuation)) for u in words_lower]):
            del candidates[k]

        # discard candidates composed of 1-2 characters
       
        elif len(''.join(words_lower)) < mininum_length:
            del candidates[k]

        # discard candidates containing small words (1-character)
        elif min([len(u) for u in words_lower]) < mininum_word_size:
            del candidates[k]

        # discard candidates composed of more than 5 words
        elif len(v.lexical_form) > maximum_word_number:
            del candidates[k]

        # discard if not containing only alpha-numeric characters
        if only_alphanum and k in candidates:
            if not all([_is_alphanum(w) for w in words_lower]):
                del self.candidates[k]

                
    ### STEP 3 : CANDIDATE WEIGHTING             
    # cluster the candidates
    topic = []
    topic = topic_clustering( candidates , threshold = 0.74 , method = 'average')
    
         
    # build the topic graph
    graph = nx.Graph()
    """ The topic graph. """
    graph = build_topic_graph(graph , topic , candidates )

   

    # compute the word scores using random walk
    w = nx.pagerank_scipy(graph, alpha=0.85, weight='weight')
    
    heuristic = None    
    weights = {}
    # loop throught the topics
    for i, topic_ in enumerate(topic):
        # get the offsets of the topic candidates
        offsets = [candidates[t].offsets for t in topic_]

        # get first candidate from topic   
        if heuristic == 'frequent':

        # get frequencies for each candidate within the topic
            freq = [len(candidates[t].surface_forms) for t in topic_]

        # get the indexes of the most frequent candidates
            indexes = [j for j, f in enumerate(freq) if f == max(freq)]

        # offsets of the indexes
            indexes_offsets = [offsets[j] for j in indexes]
            most_frequent = indexes_offsets.index(min(indexes_offsets))
            weights[topic_[most_frequent]] = w[i]

        else:
            first = offsets.index(min(offsets))
            weights[topic_[first]] = w[i]     
            ##### GET_N_GRAMS ######
            ### GET_N_BEST_GRAMS   
    
    ### STEP 4 : GET_N_BEST 
    n = 10
    redundancy_removal = False
    stemming = False
    
    # sort candidates by descending weight
    best = sorted(weights, key = weights.get, reverse=True)

    # remove redundant candidates
    if redundancy_removal:

    # initialize a new container for non redundant candidates
        non_redundant_best = []

        # loop through the best candidates
        for candidate in best:

        # test wether candidate is redundant
            if is_redundant(candidate, non_redundant_best) == True:
                continue
        # add the candidate otherwise
            non_redundant_best.append(candidate)

        # break computation if the n-best are found
            if len(non_redundant_best) >= n:
                break

        # copy non redundant candidates in best container
        best = non_redundant_best

    # get the list of best candidates as (lexical form, weight) tuples
    n_best = [( u , weights[u]) for u in best[:min(n, len(best))]]

    # replace with surface forms if no stemming
    if not stemming:
        n_best = [(' '.join(candidates[u].surface_forms[0]).lower(),
                    weights[u]) for u in best[:min(n, len(best))]]

     # return the list of best candidates
    if len(n_best) < n:
        logging.warning(
                        'Not enough candidates to choose from '
                        '({} requested, {} given)'.format(n, len(n_best)))

    
    
    return n_best



    
          

In [77]:
a = TopicRank(doc)

In [78]:
##TOPICS
a

[('blockchain', 0.10728631155393548),
 ('block chain', 0.07489499535979609),
 ('which', 0.04264999522178342),
 ('transaction data', 0.03582978216922917),
 ('consensus', 0.02865625714472804),
 ('bitcoin', 0.028261789146602483),
 ('design', 0.027888904675543134),
 ('ledger that', 0.027445012181299924),
 ('cryptocurrency bitcoin', 0.025981430652199065),
 ('use', 0.02528390393306377)]